### Hyperparameter Optimization for Top Models

Based on baseline test F2 scores from model_results.csv, we selected the top models for hyperparameter optimization:
1. **CatBoost**: 0.7152 (F2 score)
2. **Logistic Regression**: 0.7129 (F2 score)
3. **LightGBM**: 0.6859 (F2 score)
4. **Neural Network (MLP)**: 0.7366 (F2 score)

This notebook performs hyperparameter optimization for these models using F2 score as the primary metric (emphasizes recall - critical for tsunami detection).

**Note**: After optimization, results are ranked by F2 score. See the Summary section for detailed results.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_validate, cross_val_predict
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import (
    make_scorer, accuracy_score, precision_score, recall_score, 
    f1_score, fbeta_score, roc_auc_score, confusion_matrix
)
from pathlib import Path
from datetime import datetime
import lightgbm as lgb
from catboost import CatBoostClassifier
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [3]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [4]:
# Load data
data_path = Path("../data/processed/earthquake_data_tsunami_scaled.csv")
data_df = pd.read_csv(data_path)

# Prepare features (same as in previous notebooks)
features_to_exclude = ['tsunami', 'Year', 'Month','month_number','dmin','nst','longitude','latitude']
X = data_df.drop(columns=[col for col in features_to_exclude if col in data_df.columns])
y = data_df['tsunami']

In [5]:
# MLP Model Definition (from 04_G_NeuralNetwork.ipynb)
class MLP(nn.Module):
    """
    Multi-Layer Perceptron for binary classification.
    Architecture: input_dim -> hidden_dim -> hidden_dim//2 -> 1 (logits)
    Uses BCEWithLogitsLoss, so output is logits (sigmoid applied in loss function).
    """
    def __init__(self, input_dim, hidden_dim=64, dropout_rate=0.5):
        super(MLP, self).__init__()
        # First hidden layer
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        # Dropout for regularization (prevents overfitting)
        self.dropout = nn.Dropout(p=dropout_rate)
        # Second hidden layer (reduces dimensionality)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.relu2 = nn.ReLU()
        # Output layer (single neuron for binary classification)
        self.output = nn.Linear(hidden_dim // 2, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.dropout(x)  # Applied during training, disabled during eval
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.output(x)  # Returns logits (not probabilities)
        return x


# Sklearn-compatible wrapper for PyTorch MLP
class MLPClassifier(BaseEstimator, ClassifierMixin):
    """
    Sklearn-compatible wrapper for PyTorch MLP model.
    """
    def __init__(self, hidden_dim=64, dropout_rate=0.5, learning_rate=0.001, 
                 batch_size=32, epochs=100, patience=10, pos_weight=None, random_state=42):
        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.patience = patience
        self.pos_weight = pos_weight
        self.random_state = random_state
        self.model = None
        self.scaler = StandardScaler()
        self.input_dim = None
        
    def fit(self, X, y):
        # Set random seeds for reproducibility
        torch.manual_seed(self.random_state)
        np.random.seed(self.random_state)
        
        # Store input dimension
        self.input_dim = X.shape[1]
        
        # Scale features
        X_scaled = self.scaler.fit_transform(X)
        
        # Convert to PyTorch tensors
        X_tensor = torch.FloatTensor(X_scaled)
        y_tensor = torch.FloatTensor(y.values if hasattr(y, 'values') else y).unsqueeze(1)
        
        # Split into train and validation for early stopping
        X_train, X_val, y_train, y_val = train_test_split(
            X_tensor, y_tensor, test_size=0.2, random_state=self.random_state, 
            stratify=y if hasattr(y, 'values') else y
        )
        
        # Create data loader
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        
        # Initialize model
        self.model = MLP(input_dim=self.input_dim, hidden_dim=self.hidden_dim, 
                        dropout_rate=self.dropout_rate)
        
        # Setup loss and optimizer
        if self.pos_weight is not None:
            pos_weight_tensor = torch.tensor(self.pos_weight, dtype=torch.float32)
        else:
            pos_weight_tensor = torch.tensor(1.0, dtype=torch.float32)
        
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        
        # Early stopping setup
        best_val_loss = float('inf')
        patience_counter = 0
        best_model_state = None
        
        # Training loop
        for epoch in range(self.epochs):
            # Training phase
            self.model.train()
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
            
            # Validation phase
            self.model.eval()
            with torch.no_grad():
                val_outputs = self.model(X_val)
                val_loss = criterion(val_outputs, y_val).item()
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_model_state = self.model.state_dict().copy()
            else:
                patience_counter += 1
                if patience_counter >= self.patience:
                    break
        
        # Load best model
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)
        
        return self
    
    def predict_proba(self, X):
        """Return probability predictions"""
        self.model.eval()
        X_scaled = self.scaler.transform(X)
        X_tensor = torch.FloatTensor(X_scaled)
        
        with torch.no_grad():
            logits = self.model(X_tensor)
            proba = torch.sigmoid(logits).numpy().flatten()
        
        # Return in sklearn format: [prob_class_0, prob_class_1]
        return np.column_stack([1 - proba, proba])
    
    def predict(self, X):
        """Return binary predictions"""
        proba = self.predict_proba(X)[:, 1]
        return (proba > 0.5).astype(int)


In [6]:
# Setup cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)

# F2 scorer (emphasizes recall - critical for tsunami detection)
f2_scorer = make_scorer(fbeta_score, beta=2.0, zero_division=0)

# Calculate class weight ratio
class_weight_ratio = (y == 0).sum() / (y == 1).sum()
float(class_weight_ratio)

3.5454545454545454

## Model 1: CatBoost Hyperparameter Optimization


In [7]:
# CatBoost parameter grid
catboost_param_grid = {
    'depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'iterations': [100, 200, 300],
    'l2_leaf_reg': [1, 3, 5, 7],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'min_data_in_leaf': [1, 3, 5, 10],
    'random_strength': [0.5, 1.0, 2.0]
}

print("CatBoost parameter grid:")
for key, value in catboost_param_grid.items():
    print(f"  {key}: {value}")

CatBoost parameter grid:
  depth: [3, 4, 5, 6]
  learning_rate: [0.01, 0.05, 0.1, 0.15]
  iterations: [100, 200, 300]
  l2_leaf_reg: [1, 3, 5, 7]
  subsample: [0.6, 0.7, 0.8, 0.9]
  min_data_in_leaf: [1, 3, 5, 10]
  random_strength: [0.5, 1.0, 2.0]


In [8]:
# CatBoost base model
catboost_base = CatBoostClassifier(
    class_weights=[1, class_weight_ratio],
    random_state=RANDOM_STATE,
    verbose=False,
    allow_writing_files=False,
    loss_function='Logloss'
)

# Randomized search (faster than grid search for large parameter spaces)
catboost_search = RandomizedSearchCV(
    estimator=catboost_base,
    param_distributions=catboost_param_grid,
    n_iter=50,  # Number of parameter settings sampled
    cv=skf,
    scoring=f2_scorer,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1
)

catboost_search.fit(X, y)

print("\nCatBoost Best Parameters:")
print(catboost_search.best_params_)
print(f"\nCatBoost Best F2 Score: {catboost_search.best_score_:.4f}")

Fitting 5 folds for each of 50 candidates, totalling 250 fits

CatBoost Best Parameters:
{'subsample': 0.8, 'random_strength': 0.5, 'min_data_in_leaf': 3, 'learning_rate': 0.05, 'l2_leaf_reg': 3, 'iterations': 100, 'depth': 4}

CatBoost Best F2 Score: 0.7477


In [9]:
# Evaluate best CatBoost model with full metrics
scoring_dict = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'f2': f2_scorer,
    'roc_auc': make_scorer(roc_auc_score)
}

catboost_best = catboost_search.best_estimator_
catboost_cv_results = cross_validate(
    catboost_best, X, y,
    cv=skf,
    scoring=scoring_dict,
    return_train_score=True,
    n_jobs=-1
)

catboost_y_pred = cross_val_predict(catboost_best, X, y, cv=skf, n_jobs=-1)
catboost_cm = confusion_matrix(y, catboost_y_pred)
catboost_fn_rate = catboost_cm[1, 0] / catboost_cm[1, :].sum() * 100

print("CatBoost Optimized Results:")
print(f"  Test Accuracy: {catboost_cv_results['test_accuracy'].mean():.4f}")
print(f"  Test Precision: {catboost_cv_results['test_precision'].mean():.4f}")
print(f"  Test Recall: {catboost_cv_results['test_recall'].mean():.4f}")
print(f"  Test F1: {catboost_cv_results['test_f1'].mean():.4f}")
print(f"  Test F2: {catboost_cv_results['test_f2'].mean():.4f}")
print(f"  Test ROC-AUC: {catboost_cv_results['test_roc_auc'].mean():.4f}")
print(f"  False Negative Rate: {catboost_fn_rate:.2f}%")
print(f"  Train/Test Gap (Accuracy): {catboost_cv_results['train_accuracy'].mean() - catboost_cv_results['test_accuracy'].mean():.4f}")


CatBoost Optimized Results:
  Test Accuracy: 0.8300
  Test Precision: 0.5839
  Test Recall: 0.8054
  Test F1: 0.6760
  Test F2: 0.7477
  Test ROC-AUC: 0.8212
  False Negative Rate: 19.48%
  Train/Test Gap (Accuracy): 0.0293


## Model 2: Logistic Regression Hyperparameter Optimization

In [10]:
# Logistic Regression parameter grid
lr_param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
    'classifier__penalty': ['l1', 'l2', 'elasticnet'],
    'classifier__solver': ['lbfgs', 'liblinear', 'saga'],
    'classifier__max_iter': [1000, 2000, 5000],
    'classifier__class_weight': ['balanced', {0: 1, 1: class_weight_ratio}, None]
}

print("Logistic Regression parameter grid:")
for key, value in lr_param_grid.items():
    print(f"  {key}: {value}")


Logistic Regression parameter grid:
  classifier__C: [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
  classifier__penalty: ['l1', 'l2', 'elasticnet']
  classifier__solver: ['lbfgs', 'liblinear', 'saga']
  classifier__max_iter: [1000, 2000, 5000]
  classifier__class_weight: ['balanced', {0: 1, 1: np.float64(3.5454545454545454)}, None]


In [11]:
# Logistic Regression pipeline with StandardScaler
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=RANDOM_STATE))
])


lr_search = RandomizedSearchCV(
    estimator=lr_pipeline,
    param_distributions={
        'classifier__C': lr_param_grid['classifier__C'],
        'classifier__penalty': ['l1', 'l2'],  # Exclude elasticnet for simplicity
        'classifier__solver': ['liblinear', 'saga'],  # Both support l1 and l2
        'classifier__max_iter': lr_param_grid['classifier__max_iter'],
        'classifier__class_weight': lr_param_grid['classifier__class_weight']
    },
    n_iter=50,  # Sample 50 parameter combinations
    cv=skf,
    scoring=f2_scorer,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1,
    error_score='raise'
)

lr_search.fit(X, y)

print("\nLogistic Regression Best Parameters:")
print(lr_search.best_params_)
print(f"\nLogistic Regression Best F2 Score: {lr_search.best_score_:.4f}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits

Logistic Regression Best Parameters:
{'classifier__solver': 'saga', 'classifier__penalty': 'l1', 'classifier__max_iter': 1000, 'classifier__class_weight': 'balanced', 'classifier__C': 0.1}

Logistic Regression Best F2 Score: 0.7215


In [12]:
# Evaluate best Logistic Regression model
lr_best = lr_search.best_estimator_
lr_cv_results = cross_validate(
    lr_best, X, y,
    cv=skf,
    scoring=scoring_dict,
    return_train_score=True,
    n_jobs=-1
)

lr_y_pred = cross_val_predict(lr_best, X, y, cv=skf, n_jobs=-1)
lr_cm = confusion_matrix(y, lr_y_pred)
lr_fn_rate = lr_cm[1, 0] / lr_cm[1, :].sum() * 100

print("Logistic Regression Optimized Results:")
print(f"  Test Accuracy: {lr_cv_results['test_accuracy'].mean():.4f}")
print(f"  Test Precision: {lr_cv_results['test_precision'].mean():.4f}")
print(f"  Test Recall: {lr_cv_results['test_recall'].mean():.4f}")
print(f"  Test F1: {lr_cv_results['test_f1'].mean():.4f}")
print(f"  Test F2: {lr_cv_results['test_f2'].mean():.4f}")
print(f"  Test ROC-AUC: {lr_cv_results['test_roc_auc'].mean():.4f}")
print(f"  False Negative Rate: {lr_fn_rate:.2f}%")
print(f"  Train/Test Gap (Accuracy): {lr_cv_results['train_accuracy'].mean() - lr_cv_results['test_accuracy'].mean():.4f}")

Logistic Regression Optimized Results:
  Test Accuracy: 0.7729
  Test Precision: 0.4933
  Test Recall: 0.8187
  Test F1: 0.6139
  Test F2: 0.7215
  Test ROC-AUC: 0.7894
  False Negative Rate: 18.18%
  Train/Test Gap (Accuracy): 0.0039


## Model 3: LightGBM Hyperparameter Optimization

In [13]:
# LightGBM parameter grid
lgbm_param_grid = {
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'n_estimators': [100, 200, 300, 400],
    'num_leaves': [15, 31, 50, 70],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'min_child_samples': [5, 10, 20, 30],
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    'reg_lambda': [0, 0.1, 0.5, 1.0]
}

print("LightGBM parameter grid:")
for key, value in lgbm_param_grid.items():
    print(f"  {key}: {value}")

LightGBM parameter grid:
  max_depth: [3, 4, 5, 6, 7]
  learning_rate: [0.01, 0.05, 0.1, 0.15]
  n_estimators: [100, 200, 300, 400]
  num_leaves: [15, 31, 50, 70]
  subsample: [0.6, 0.7, 0.8, 0.9]
  colsample_bytree: [0.6, 0.7, 0.8, 0.9]
  min_child_samples: [5, 10, 20, 30]
  reg_alpha: [0, 0.1, 0.5, 1.0]
  reg_lambda: [0, 0.1, 0.5, 1.0]


In [14]:
# LightGBM base model
lgbm_base = lgb.LGBMClassifier(
    scale_pos_weight=class_weight_ratio,
    random_state=RANDOM_STATE,
    verbosity=-1,
    force_col_wise=True,
    objective='binary',
    metric='binary_logloss'
)

# Randomized search
lgbm_search = RandomizedSearchCV(
    estimator=lgbm_base,
    param_distributions=lgbm_param_grid,
    n_iter=50,  # Number of parameter settings sampled
    cv=skf,
    scoring=f2_scorer,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1
)

lgbm_search.fit(X, y)

print("\nLightGBM Best Parameters:")
print(lgbm_search.best_params_)
print(f"\nLightGBM Best F2 Score: {lgbm_search.best_score_:.4f}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits

LightGBM Best Parameters:
{'subsample': 0.7, 'reg_lambda': 0.1, 'reg_alpha': 0.5, 'num_leaves': 15, 'n_estimators': 400, 'min_child_samples': 20, 'max_depth': 4, 'learning_rate': 0.01, 'colsample_bytree': 0.9}

LightGBM Best F2 Score: 0.7609


In [15]:
# Evaluate best LightGBM model
lgbm_best = lgbm_search.best_estimator_
lgbm_cv_results = cross_validate(
    lgbm_best, X, y,
    cv=skf,
    scoring=scoring_dict,
    return_train_score=True,
    n_jobs=-1
)

lgbm_y_pred = cross_val_predict(lgbm_best, X, y, cv=skf, n_jobs=-1)
lgbm_cm = confusion_matrix(y, lgbm_y_pred)
lgbm_fn_rate = lgbm_cm[1, 0] / lgbm_cm[1, :].sum() * 100

print("LightGBM Optimized Results:")
print(f"  Test Accuracy: {lgbm_cv_results['test_accuracy'].mean():.4f}")
print(f"  Test Precision: {lgbm_cv_results['test_precision'].mean():.4f}")
print(f"  Test Recall: {lgbm_cv_results['test_recall'].mean():.4f}")
print(f"  Test F1: {lgbm_cv_results['test_f1'].mean():.4f}")
print(f"  Test F2: {lgbm_cv_results['test_f2'].mean():.4f}")
print(f"  Test ROC-AUC: {lgbm_cv_results['test_roc_auc'].mean():.4f}")
print(f"  False Negative Rate: {lgbm_fn_rate:.2f}%")
print(f"  Train/Test Gap (Accuracy): {lgbm_cv_results['train_accuracy'].mean() - lgbm_cv_results['test_accuracy'].mean():.4f}")

LightGBM Optimized Results:
  Test Accuracy: 0.8300
  Test Precision: 0.5828
  Test Recall: 0.8245
  Test F1: 0.6823
  Test F2: 0.7609
  Test ROC-AUC: 0.8280
  False Negative Rate: 17.53%
  Train/Test Gap (Accuracy): 0.0514


## Comparison of Optimized Models

## Model 4: Neural Network (MLP) Hyperparameter Optimization

In [16]:
# Neural Network (MLP) parameter grid
mlp_param_grid = {
    'hidden_dim': [32, 64, 128],
    'dropout_rate': [0.3, 0.4, 0.5, 0.6],
    'learning_rate': [0.0001, 0.001, 0.01],
    'batch_size': [16, 32, 64],
    'epochs': [50, 100, 150],
    'patience': [5, 10, 15]
}

print("Neural Network (MLP) parameter grid:")
for key, value in mlp_param_grid.items():
    print(f"  {key}: {value}")

Neural Network (MLP) parameter grid:
  hidden_dim: [32, 64, 128]
  dropout_rate: [0.3, 0.4, 0.5, 0.6]
  learning_rate: [0.0001, 0.001, 0.01]
  batch_size: [16, 32, 64]
  epochs: [50, 100, 150]
  patience: [5, 10, 15]


In [17]:
# Neural Network base model
mlp_base = MLPClassifier(
    pos_weight=class_weight_ratio,
    random_state=RANDOM_STATE
)

# Randomized search (reduced n_iter for neural networks due to longer training time)
mlp_search = RandomizedSearchCV(
    estimator=mlp_base,
    param_distributions=mlp_param_grid,
    n_iter=30,  # Reduced from 50 due to longer training time for neural networks
    cv=skf,
    scoring=f2_scorer,
    n_jobs=1,  # Neural networks don't benefit from parallelization in this context
    random_state=RANDOM_STATE,
    verbose=1
)

mlp_search.fit(X, y)

print("\nNeural Network (MLP) Best Parameters:")
print(mlp_search.best_params_)
print(f"\nNeural Network (MLP) Best F2 Score: {mlp_search.best_score_:.4f}")


Fitting 5 folds for each of 30 candidates, totalling 150 fits

Neural Network (MLP) Best Parameters:
{'patience': 5, 'learning_rate': 0.01, 'hidden_dim': 32, 'epochs': 50, 'dropout_rate': 0.5, 'batch_size': 16}

Neural Network (MLP) Best F2 Score: 0.7413


In [18]:
# Evaluate best Neural Network model
mlp_best = mlp_search.best_estimator_
mlp_cv_results = cross_validate(
    mlp_best, X, y,
    cv=skf,
    scoring=scoring_dict,
    return_train_score=True,
    n_jobs=1  # Neural networks don't benefit from parallelization
)

mlp_y_pred = cross_val_predict(mlp_best, X, y, cv=skf, n_jobs=1)
mlp_cm = confusion_matrix(y, mlp_y_pred)
mlp_fn_rate = mlp_cm[1, 0] / mlp_cm[1, :].sum() * 100

print("Neural Network (MLP) Optimized Results:")
print(f"  Test Accuracy: {mlp_cv_results['test_accuracy'].mean():.4f}")
print(f"  Test Precision: {mlp_cv_results['test_precision'].mean():.4f}")
print(f"  Test Recall: {mlp_cv_results['test_recall'].mean():.4f}")
print(f"  Test F1: {mlp_cv_results['test_f1'].mean():.4f}")
print(f"  Test F2: {mlp_cv_results['test_f2'].mean():.4f}")
print(f"  Test ROC-AUC: {mlp_cv_results['test_roc_auc'].mean():.4f}")
print(f"  False Negative Rate: {mlp_fn_rate:.2f}%")
print(f"  Train/Test Gap (Accuracy): {mlp_cv_results['train_accuracy'].mean() - mlp_cv_results['test_accuracy'].mean():.4f}")


Neural Network (MLP) Optimized Results:
  Test Accuracy: 0.8000
  Test Precision: 0.5324
  Test Recall: 0.8252
  Test F1: 0.6449
  Test F2: 0.7413
  Test ROC-AUC: 0.8091
  False Negative Rate: 17.53%
  Train/Test Gap (Accuracy): 0.0100


In [19]:
# Create comparison DataFrame
comparison_data = {
    'Model': ['CatBoost (Optimized)', 'Logistic Regression (Optimized)', 'LightGBM (Optimized)', 'Neural Network (MLP) (Optimized)'],
    'Test Accuracy': [
        catboost_cv_results['test_accuracy'].mean(),
        lr_cv_results['test_accuracy'].mean(),
        lgbm_cv_results['test_accuracy'].mean(),
        mlp_cv_results['test_accuracy'].mean()
    ],
    'Test Precision': [
        catboost_cv_results['test_precision'].mean(),
        lr_cv_results['test_precision'].mean(),
        lgbm_cv_results['test_precision'].mean(),
        mlp_cv_results['test_precision'].mean()
    ],
    'Test Recall': [
        catboost_cv_results['test_recall'].mean(),
        lr_cv_results['test_recall'].mean(),
        lgbm_cv_results['test_recall'].mean(),
        mlp_cv_results['test_recall'].mean()
    ],
    'Test F1': [
        catboost_cv_results['test_f1'].mean(),
        lr_cv_results['test_f1'].mean(),
        lgbm_cv_results['test_f1'].mean(),
        mlp_cv_results['test_f1'].mean()
    ],
    'Test F2': [
        catboost_cv_results['test_f2'].mean(),
        lr_cv_results['test_f2'].mean(),
        lgbm_cv_results['test_f2'].mean(),
        mlp_cv_results['test_f2'].mean()
    ],
    'Test ROC-AUC': [
        catboost_cv_results['test_roc_auc'].mean(),
        lr_cv_results['test_roc_auc'].mean(),
        lgbm_cv_results['test_roc_auc'].mean(),
        mlp_cv_results['test_roc_auc'].mean()
    ],
    'False Negative Rate (%)': [catboost_fn_rate, lr_fn_rate, lgbm_fn_rate, mlp_fn_rate],
    'Train/Test Gap (Accuracy)': [
        catboost_cv_results['train_accuracy'].mean() - catboost_cv_results['test_accuracy'].mean(),
        lr_cv_results['train_accuracy'].mean() - lr_cv_results['test_accuracy'].mean(),
        lgbm_cv_results['train_accuracy'].mean() - lgbm_cv_results['test_accuracy'].mean(),
        mlp_cv_results['train_accuracy'].mean() - mlp_cv_results['test_accuracy'].mean()
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.round(4)
print("\n" + "="*80)
print("COMPARISON OF OPTIMIZED MODELS")
print("="*80)
print(comparison_df.to_string(index=False))

# Sort by F2 score (primary metric)
print("\n" + "="*80)
print("MODELS RANKED BY TEST F2 SCORE (Primary Metric)")
print("="*80)
print(comparison_df.sort_values('Test F2', ascending=False).to_string(index=False))



COMPARISON OF OPTIMIZED MODELS
                           Model  Test Accuracy  Test Precision  Test Recall  Test F1  Test F2  Test ROC-AUC  False Negative Rate (%)  Train/Test Gap (Accuracy)
            CatBoost (Optimized)         0.8300          0.5839       0.8054   0.6760   0.7477        0.8212                  19.4805                     0.0293
 Logistic Regression (Optimized)         0.7729          0.4933       0.8187   0.6139   0.7215        0.7894                  18.1818                     0.0039
            LightGBM (Optimized)         0.8300          0.5828       0.8245   0.6823   0.7609        0.8280                  17.5325                     0.0514
Neural Network (MLP) (Optimized)         0.8000          0.5324       0.8252   0.6449   0.7413        0.8091                  17.5325                     0.0100

MODELS RANKED BY TEST F2 SCORE (Primary Metric)
                           Model  Test Accuracy  Test Precision  Test Recall  Test F1  Test F2  Test ROC-AUC  Fals

In [ ]:
# Add Neural Network results to best_params (if not already included)
if 'neural_network_mlp' not in best_params:
    best_params['neural_network_mlp'] = {
        'best_params': mlp_search.best_params_,
        'best_f2_score': float(mlp_search.best_score_),
        'test_metrics': {
            'accuracy': float(mlp_cv_results['test_accuracy'].mean()),
            'precision': float(mlp_cv_results['test_precision'].mean()),
            'recall': float(mlp_cv_results['test_recall'].mean()),
            'f1': float(mlp_cv_results['test_f1'].mean()),
            'f2': float(mlp_cv_results['test_f2'].mean()),
            'roc_auc': float(mlp_cv_results['test_roc_auc'].mean()),
            'false_negative_rate': float(mlp_fn_rate)
        }
    }
    # Re-save JSON file with Neural Network results
    with open(params_file, 'w') as f:
        json.dump(best_params, f, indent=4)
    print("Neural Network results added to best_hyperparameters.json")
else:
    print("Neural Network results already in best_hyperparameters.json")


## Save Best Parameters and Results

In [20]:
# Save best parameters to JSON
best_params = {
    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'catboost': {
        'best_params': catboost_search.best_params_,
        'best_f2_score': float(catboost_search.best_score_),
        'test_metrics': {
            'accuracy': float(catboost_cv_results['test_accuracy'].mean()),
            'precision': float(catboost_cv_results['test_precision'].mean()),
            'recall': float(catboost_cv_results['test_recall'].mean()),
            'f1': float(catboost_cv_results['test_f1'].mean()),
            'f2': float(catboost_cv_results['test_f2'].mean()),
            'roc_auc': float(catboost_cv_results['test_roc_auc'].mean()),
            'false_negative_rate': float(catboost_fn_rate)
        }
    },
    'logistic_regression': {
        'best_params': {k.replace('classifier__', ''): v for k, v in lr_search.best_params_.items()},
        'best_f2_score': float(lr_search.best_score_),
        'test_metrics': {
            'accuracy': float(lr_cv_results['test_accuracy'].mean()),
            'precision': float(lr_cv_results['test_precision'].mean()),
            'recall': float(lr_cv_results['test_recall'].mean()),
            'f1': float(lr_cv_results['test_f1'].mean()),
            'f2': float(lr_cv_results['test_f2'].mean()),
            'roc_auc': float(lr_cv_results['test_roc_auc'].mean()),
            'false_negative_rate': float(lr_fn_rate)
        }
    },
    'lightgbm': {
        'best_params': lgbm_search.best_params_,
        'best_f2_score': float(lgbm_search.best_score_),
        'test_metrics': {
            'accuracy': float(lgbm_cv_results['test_accuracy'].mean()),
            'precision': float(lgbm_cv_results['test_precision'].mean()),
            'recall': float(lgbm_cv_results['test_recall'].mean()),
            'f1': float(lgbm_cv_results['test_f1'].mean()),
            'f2': float(lgbm_cv_results['test_f2'].mean()),
            'roc_auc': float(lgbm_cv_results['test_roc_auc'].mean()),
            'false_negative_rate': float(lgbm_fn_rate)
        }
    }
}

# Save to JSON file
results_dir = Path("../models")
results_dir.mkdir(parents=True, exist_ok=True)
params_file = results_dir / "best_hyperparameters.json"

with open(params_file, 'w') as f:
    json.dump(best_params, f, indent=4)

print("\nBest Parameters Summary:")
for model_name, model_data in best_params.items():
    if model_name != 'timestamp':
        print(f"\n{model_name.upper().replace('_', ' ')}:")
        print(f"  Best F2 Score: {model_data['best_f2_score']:.4f}")
        print(f"  Best Parameters:")
        for param, value in model_data['best_params'].items():
            print(f"    {param}: {value}")



Best Parameters Summary:

CATBOOST:
  Best F2 Score: 0.7477
  Best Parameters:
    subsample: 0.8
    random_strength: 0.5
    min_data_in_leaf: 3
    learning_rate: 0.05
    l2_leaf_reg: 3
    iterations: 100
    depth: 4

LOGISTIC REGRESSION:
  Best F2 Score: 0.7215
  Best Parameters:
    solver: saga
    penalty: l1
    max_iter: 1000
    class_weight: balanced
    C: 0.1

LIGHTGBM:
  Best F2 Score: 0.7609
  Best Parameters:
    subsample: 0.7
    reg_lambda: 0.1
    reg_alpha: 0.5
    num_leaves: 15
    n_estimators: 400
    min_child_samples: 20
    max_depth: 4
    learning_rate: 0.01
    colsample_bytree: 0.9


In [21]:
# Save optimized results to CSV (append to existing model_results.csv)
results_csv = results_dir / "model_results.csv"

# Prepare results for CSV
optimized_results = []

for model_name, model_data, cv_results, fn_rate in [
    ('CatBoost (Optimized)', 'CatBoost', catboost_cv_results, catboost_fn_rate),
    ('Logistic Regression (Optimized)', 'Logistic Regression', lr_cv_results, lr_fn_rate),
    ('LightGBM (Optimized)', 'LightGBM', lgbm_cv_results, lgbm_fn_rate),
    ('Neural Network (MLP) (Optimized)', 'Neural Network (MLP)', mlp_cv_results, mlp_fn_rate)
]:
    result = {
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'model': model_name,
        'cv_splits': n_splits,
        'scaler': 'StandardScaler' if 'Logistic' in model_name or 'Neural Network' in model_name else 'PowerTransformer+StandardScaler',
        'class_weight': f"class_weights=[1, {class_weight_ratio:.2f}]" if 'CatBoost' in model_name else (
            'balanced' if 'Logistic' in model_name else (
                f"pos_weight={class_weight_ratio:.2f}" if 'Neural Network' in model_name else f"scale_pos_weight={class_weight_ratio:.2f}"
            )
        ),
        'test_accuracy': cv_results['test_accuracy'].mean(),
        'test_precision': cv_results['test_precision'].mean(),
        'test_recall': cv_results['test_recall'].mean(),
        'test_f1': cv_results['test_f1'].mean(),
        'test_f2': cv_results['test_f2'].mean(),
        'test_roc_auc': cv_results['test_roc_auc'].mean(),
        'train_accuracy': cv_results['train_accuracy'].mean(),
        'train_precision': cv_results['train_precision'].mean(),
        'train_recall': cv_results['train_recall'].mean(),
        'train_f1': cv_results['train_f1'].mean(),
        'train_f2': cv_results['train_f2'].mean(),
        'train_roc_auc': cv_results['train_roc_auc'].mean(),
        'train_test_gap_accuracy': cv_results['train_accuracy'].mean() - cv_results['test_accuracy'].mean(),
        'false_negative_rate': fn_rate,
        'false_negative_percentage': (fn_rate / 100) * (y == 1).sum() / len(y) * 100,
        'notes': f"Optimized hyperparameters - see best_hyperparameters.json"
    }
    optimized_results.append(result)

# Append to existing CSV
optimized_df = pd.DataFrame(optimized_results)
if results_csv.exists():
    existing_results = pd.read_csv(results_csv)
    all_results = pd.concat([existing_results, optimized_df], ignore_index=True)
    all_results.to_csv(results_csv, index=False)
else:
    optimized_df.to_csv(results_csv, index=False)

## Summary

Hyperparameter optimization has been completed for the top models based on F2 score. Results are ranked by optimized F2 score:

### Optimization Results (Ranked by F2 Score):

1. **LightGBM (Optimized)** - **BEST PERFORMER**
   - **Test F2 Score**: 0.7609 (↑ from 0.6859 baseline, +10.9% improvement)
   - **Test Accuracy**: 0.8300 (↑ from 0.8357 baseline)
   - **Test Recall**: 0.8245 (↑ from 0.7080 baseline, +16.5% improvement)
   - **Test Precision**: 0.5828
   - **Test ROC-AUC**: 0.8280 (↑ from 0.7899 baseline)
   - **False Negative Rate**: 17.53% (↓ from 29.22% baseline, -40% reduction) - **LOWEST**
   - **Train/Test Gap**: 0.0514 (5.14%) - Moderate overfitting
   - **Key Hyperparameters**: learning_rate=0.01, n_estimators=400, max_depth=4, num_leaves=15

2. **CatBoost (Optimized)**
   - **Test F2 Score**: 0.7477 (↑ from 0.7152 baseline, +4.5% improvement)
   - **Test Accuracy**: 0.8300 (↑ from 0.8186 baseline)
   - **Test Recall**: 0.8054 (↑ from 0.7662 baseline, +5.1% improvement)
   - **Test Precision**: 0.5839
   - **Test ROC-AUC**: 0.8212 (↓ from 0.8999 baseline)
   - **False Negative Rate**: 19.48% (↓ from 23.38% baseline, -16.7% reduction)
   - **Train/Test Gap**: 0.0293 (2.93%) - Minimal overfitting
   - **Key Hyperparameters**: learning_rate=0.05, iterations=100, depth=4, subsample=0.8

3. **Neural Network (MLP) (Optimized)**
   - **Test F2 Score**: 0.7413 (↑ from 0.7366 baseline, +0.6% improvement)
   - **Test Accuracy**: 0.8000 (↓ from 0.8114 baseline)
   - **Test Recall**: 0.8252 (↑ from 0.8052 baseline, +2.5% improvement)
   - **Test Precision**: 0.5324
   - **Test ROC-AUC**: 0.8091 (↓ from 0.8817 baseline)
   - **False Negative Rate**: 17.53% (same as baseline) - **Tied for LOWEST**
   - **Train/Test Gap**: 0.0100 (1.00%) - Excellent generalization
   - **Key Hyperparameters**: hidden_dim=32, dropout_rate=0.5, learning_rate=0.01, batch_size=16, epochs=50, patience=5

4. **Logistic Regression (Optimized)**
   - **Test F2 Score**: 0.7215 (↑ from 0.7129 baseline, +1.2% improvement)
   - **Test Accuracy**: 0.7729 (↓ from 0.7800 baseline)
   - **Test Recall**: 0.8187 (↑ from 0.7987 baseline, +2.5% improvement)
   - **Test Precision**: 0.4933
   - **Test ROC-AUC**: 0.7894 (↑ from 0.7867 baseline)
   - **False Negative Rate**: 18.18% (↓ from 20.13% baseline, -9.7% reduction)
   - **Train/Test Gap**: 0.0039 (0.39%) - **NO OVERFITTING** - Best generalization
   - **Key Hyperparameters**: C=0.1, penalty='l1', solver='saga', class_weight='balanced'

### Key Findings:

- **LightGBM achieved the best F2 score (0.7609)** and tied for lowest false negative rate (17.53%)
- **Neural Network (MLP)** achieved the second-best recall (0.8252) and tied for lowest false negative rate (17.53%)
- All optimized models showed **improved or maintained F2 scores** compared to baseline models
- **False negative rates decreased or maintained** for all models, which is critical for tsunami detection
- **Logistic Regression** shows the best generalization with minimal train/test gap (0.39%)
- **Neural Network (MLP)** shows excellent generalization with only 1.00% train/test gap
- All models were optimized using **F2 score** as the primary metric (emphasizes recall - critical for tsunami detection)
- 5-fold stratified cross-validation was used to ensure robust evaluation
- RandomizedSearchCV was used for efficiency, sampling 30-50 parameter combinations per model (30 for Neural Network due to longer training time)
- Best hyperparameters have been saved to `models/best_hyperparameters.json`
- Optimized results have been appended to `models/model_results.csv`

### Model Comparison:

| Metric | LightGBM (Opt) | CatBoost (Opt) | Neural Network (Opt) | Logistic Reg (Opt) | Winner |
|--------|----------------|----------------|---------------------|-------------------|--------|
| **F2 Score** | 0.7609 | 0.7477 | 0.7413 | 0.7215 | **LightGBM** |
| **Recall** | 0.8245 | 0.8054 | 0.8252 | 0.8187 | **Neural Network** |
| **False Negative Rate** | 17.53% | 19.48% | 17.53% | 18.18% | **LightGBM/Neural Network** |
| **Accuracy** | 0.8300 | 0.8300 | 0.8000 | 0.7729 | **LightGBM/CatBoost** |
| **ROC-AUC** | 0.8280 | 0.8212 | 0.8091 | 0.7894 | **LightGBM** |
| **Generalization** | 5.14% gap | 2.93% gap | 1.00% gap | 0.39% gap | **Logistic Reg** |

### Recommendations:

1. **For Production Deployment**: **LightGBM (Optimized)** - Best overall performance with highest F2 score (0.7609) and tied for lowest false negative rate (17.53%)
2. **For High Recall Requirements**: **Neural Network (MLP) (Optimized)** - Highest recall (0.8252) and tied for lowest false negative rate (17.53%) with excellent generalization (1.00% gap)
3. **For Interpretability**: **Logistic Regression (Optimized)** - Best generalization (0.39% gap) and interpretable coefficients
4. **For Balanced Performance**: **CatBoost (Optimized)** - Good balance between performance and overfitting control

### Next Steps:
- Deploy **LightGBM (Optimized)** as the primary model for tsunami detection
- Consider **Neural Network (MLP) (Optimized)** as an alternative for scenarios requiring maximum recall
- Consider ensemble methods combining LightGBM, Neural Network, and CatBoost for improved robustness
- Monitor false negative rate in production to ensure safety standards are met (target: <20%)